In [19]:
import numpy as np
import time
from matplotlib import pyplot as plt
import NEON_AOP_waveform_tools #importing the functions
import sys

In [18]:
#define the PLS file fo the pulswaves data
plsFile = './2015032618_Q780_150326_194954_1_originalpoints.pls'

#set the first and last numbers in the range - in the bash script set the beginning and end
#call the script using this: python [name_of_script].py begin end
begin = int(sys.argv[1])
end = int(sys.arv[2])

for pulse in range(begin,end):
    pulse_number = [pulse]
    coord_info, wave_in, wave_out = NEON_AOP_waveform_tools.waveform_read(pulse_number, plsFile)
    #print(coord_info) 
    #get the initial parameters
    initial_params_out = NEON_AOP_waveform_tools.get_params(wave_out[0])
    initial_params_in = NEON_AOP_waveform_tools.get_params(wave_in[0])
    
    num_peaks_in = np.array(initial_params_in[0,-1]) #the last one is the number of peaks
    WFtime = np.arange(1,244)
    temp_in_params = NEON_AOP_waveform_tools.get_gaussian_params(initial_params_in, wave_in[0],10,20,0)[0]
    temp_out_params = NEON_AOP_waveform_tools.get_gaussian_params(initial_params_out, wave_out[0],10,20,0)[0]

    fit_wave_final = NEON_AOP_waveform_tools.get_fit_wave(temp_in_params, 244, num_peaks_in)
    FWHM = (2.355*(temp_in_params[0,2:int(num_peaks_in*3):3]/2**0.5))/2
    
    trig_loc = temp_in_params[0,1:int(num_peaks_in*3):3]-FWHM-temp_out_params[0,1] #subtracting the outgoing pulse, need to normalize trigger locations by the locations of the outgoing pulse
    trig_loc = np.reshape(trig_loc,(int(num_peaks_in),1))

    pulse_coords = (np.matlib.repmat(trig_loc[:,0],3,1)).transpose()*coord_info[:,1:4]+coord_info[:,4:7]
    print(pulse_coords)



read 0 pulses in file in 0.00025199999998903877 seconds, we are [0.]% done
[[5.31524630e+05 4.31784554e+06 2.26043944e+03]
 [5.31524284e+05 4.31784562e+06 2.25923269e+03]]
[[5.31525017e+05 4.31784554e+06 2.26035814e+03]
 [5.31524672e+05 4.31784561e+06 2.25915697e+03]]
[[5.31526003e+05 4.31784506e+06 2.26212251e+03]
 [5.31525481e+05 4.31784517e+06 2.26030009e+03]
 [5.31525031e+05 4.31784527e+06 2.25872930e+03]]
1
[[5.31524311e+05 4.31784454e+06 2.26033965e+03]
 [5.31523893e+05 4.31784463e+06 2.25888468e+03]
 [5.31523628e+05 4.31784469e+06 2.25796365e+03]]
[[5.31524704e+05 4.31784454e+06 2.26027459e+03]
 [5.31524340e+05 4.31784462e+06 2.25900805e+03]
 [5.31523998e+05 4.31784469e+06 2.25781532e+03]]
1
[[5.31525244e+05 4.31784460e+06 2.26034030e+03]
 [5.31524774e+05 4.31784470e+06 2.25869973e+03]
 [5.31524495e+05 4.31784476e+06 2.25772661e+03]]
[[5.31525577e+05 4.31784457e+06 2.26015069e+03]
 [5.31525216e+05 4.31784465e+06 2.25889086e+03]
 [5.31524796e+05 4.31784474e+06 2.25742391e+03]]
[[